# 🍃 Mental Health Data Analysis

## 1. Introduction
This notebook is part of the data preparation process for a provincial-level mental health dashboard.  
The raw datasets from multiple years have been cleaned, standardized, and merged to ensure consistency and usability.  
Key transformations include column removal, renaming, and the insertion of a `pro_code` column for geographic mapping.

The final output will support visualizations such as choropleth maps, heatmaps, and KPI cards, with year-wise and province-level filtering for interactive exploration.

## 2. Data Preparation

### 2.1 Import & Configs

In [1]:
# Import libraries
import pandas as pd

# Create a list of file paths
file_info = [
    {"filename": r"data/report_2563.csv", "year": "2563"},
    {"filename": r"data/report_2564.csv", "year": "2564"},
    {"filename": r"data/report_2565.csv", "year": "2565"},
    {"filename": r"data/report_2566.csv", "year": "2566"}
]

province_code = "data/province_code.csv"

### 2.2 Dataset Overview & Loading
Mental Health Dataset: `report_2563.csv`, `report_2564.csv`, `report_2565.csv`, `report_2566.csv`

This dataset is sourced from the [Department of Mental Health](https://dmh.go.th/report/datacenter/hdc/)

Including features such as:
- จังหวัด, โรคสมองเสื่อม, ติดแอลกอฮอล์, ติดยาบ้า (Amphetamine), ติดสารเสพติดอื่นๆ, 
- โรคจิตเภท, โรคจิตอื่นๆ, โรคอารมณ์สองขั้ว, โรคชึมเศร้า, โรควิตกกังวล, ความบกพร่องทางสติปัญญา, 
- ความบกพร่องทางการเรียนรู้, โรคออทิสติก, โรคสมาธิสั้น, พยามยามฆ่าตัวตาย (การตั้งใจทำร้ายตนเอง),
- ผู้ป่วยติดเกมส์ในผู้ใหญ่ (15 ปีขึ้นไป), ผู้ป่วยติดเกมส์ในเด็ก (อายุต่ำกว่า 15 ปี), 
- โรคทางจิตเวชอื่นๆ, โรคลมชัก, รวม, ปี

Province Code Dataset: `province_code.csv`, including features such as:
- จังหวัด, pro_code


#### Mental Health Dataset

In [2]:
# Read Report
def read_report_csv(filename, year):
    try:
        # Read *.csv
        df = pd.read_csv(filename)
        # Insert column ["ปี"] and convert dtype to integer
        df["ปี"] = year
        df["ปี"] = pd.to_numeric(df["ปี"], errors = "coerce").astype("Int64")
        
        return df

    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")
    
df_list = [read_report_csv(file["filename"], file["year"])for file in file_info]

print("Sample Dataset Overview (report_2563)")
df_list[0].head()


Sample Dataset Overview (report_2563)


,จังหวัด,โรคสมองเสื่อม,ติดแอลกอฮอล์,ติดยาบ้า (Amphetamine),ติดสารเสพติดอื่นๆ,โรคจิตเภท,โรคจิตอื่นๆ,โรคอารมณ์สองขั้ว,โรคชึมเศร้า,โรควิตกกังวล,...,ความบกพร่องทางการเรียนรู้,โรคออทิสติก,โรคสมาธิสั้น,พยามยามฆ่าตัวตาย (การตั้งใจทำร้ายตนเอง),ผู้ป่วยติดเกมส์ในผู้ใหญ่ (15 ปีขึ้นไป),ผู้ป่วยติดเกมส์ในเด็ก (อายุต่ำกว่า 15 ปี),โรคทางจิตเวชอื่นๆ,โรคลมชัก,รวม,ปี
0,กรุงเทพมหานคร,"1,018",365,359,468,"1,967",412,437,"12,215","2,686",...,508,"1,195","2,351",665,28,48,"3,600","1,884","31,340",2563
1,สมุทรปราการ,146,326,435,198,888,227,46,"4,839","1,095",...,181,166,804,405,0,6,"1,631",741,"12,272",2563
2,นนทบุรี,630,468,429,787,"1,636",348,339,"3,917","2,555",...,303,395,"1,162",343,7,12,"4,403","1,410","19,656",2563
3,ปทุมธานี,179,215,493,423,"1,439",409,84,"3,060","1,603",...,139,224,440,179,4,3,"3,123",783,"13,013",2563
4,พระนครศรีอยุธยา,78,375,597,680,720,207,44,"3,332",878,...,57,60,134,240,2,0,"1,841",419,"9,716",2563


In [3]:
# Dataset Information
print("Sample Dataset Information (report_2563) \n")
df_list[0].info()

Sample Dataset Information (report_2563) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 21 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   จังหวัด                                    79 non-null     object
 1   โรคสมองเสื่อม                              79 non-null     object
 2   ติดแอลกอฮอล์                               79 non-null     object
 3   ติดยาบ้า (Amphetamine)                     79 non-null     object
 4   ติดสารเสพติดอื่นๆ                          79 non-null     object
 5   โรคจิตเภท                                  79 non-null     object
 6   โรคจิตอื่นๆ                                79 non-null     object
 7   โรคอารมณ์สองขั้ว                           79 non-null     object
 8   โรคชึมเศร้า                                79 non-null     object
 9   โรควิตกกังวล                               79 non-null     object
 1

In [4]:
# Dataset Shape
def dataset_shape():
    print("Dataset Shape \n")
    try:
        for i, year in enumerate([2563, 2564, 2565, 2566]):
            print(f"Report_{year}: {df_list[i].shape}")
    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")

dataset_shape()

Dataset Shape 

Report_2563: (79, 21)
Report_2564: (79, 21)
Report_2565: (79, 21)
Report_2566: (79, 21)


In [5]:
# Dataset Columns
print("Sample Dataset Columns (report_2563) \n")
df_list[0].columns

Sample Dataset Columns (report_2563) 



Index(['จังหวัด', 'โรคสมองเสื่อม', 'ติดแอลกอฮอล์', 'ติดยาบ้า (Amphetamine)',
       'ติดสารเสพติดอื่นๆ', 'โรคจิตเภท', 'โรคจิตอื่นๆ', 'โรคอารมณ์สองขั้ว',
       'โรคชึมเศร้า', 'โรควิตกกังวล', 'ความบกพร่องทางสติปัญญา',
       'ความบกพร่องทางการเรียนรู้', 'โรคออทิสติก', 'โรคสมาธิสั้น',
       'พยามยามฆ่าตัวตาย (การตั้งใจทำร้ายตนเอง)',
       'ผู้ป่วยติดเกมส์ในผู้ใหญ่ (15 ปีขึ้นไป)',
       'ผู้ป่วยติดเกมส์ในเด็ก (อายุต่ำกว่า 15 ปี)', 'โรคทางจิตเวชอื่นๆ',
       'โรคลมชัก', 'รวม', 'ปี'],
      dtype='object')

#### Province Code Dataset

In [6]:
# Read province_code
def load_province_metadata(filepath):
    try:
        return pd.read_csv(filepath)
    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")

df_p_code = load_province_metadata(province_code)

print("Province Code Overview")
df_p_code.head()

Province Code Overview


,จังหวัด,pro_code
0,กรุงเทพมหานคร,10
1,สมุทรปราการ,11
2,นนทบุรี,12
3,ปทุมธานี,13
4,พระนครศรีอยุธยา,14


In [7]:
# Preview infomation of province_code
print("Province Code Infomation \n")
df_p_code.info()

Province Code Infomation 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   จังหวัด   77 non-null     object
 1   pro_code  77 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.3+ KB


In [8]:
# Province Code Shape
print("Province Code Shape: ", df_p_code.shape)

Province Code Shape:  (77, 2)


In [9]:
# Province Code Columns
print("Province Code Columns: ")
list(df_p_code.columns)

Province Code Columns: 


['จังหวัด', 'pro_code']

### 2.3 Data Cleaning and Processing

#### Step 1: Missing Values

In [10]:
# Missing Values Function
def missing_values():
    print("Results of Missing Values")
    print("_________________________")
    try:
        for i, year in enumerate([2563, 2564, 2565, 2566]):
            missing = df_list[i].isnull().sum().sum()
            print(f"Report_{year}: ", missing)

        print("Province Code: ", df_p_code.isnull().sum().sum())

    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")
        
missing_values()

Results of Missing Values
_________________________
Report_2563:  0
Report_2564:  0
Report_2565:  0
Report_2566:  0
Province Code:  0


#### Step 2: Duplicate Values

In [11]:
# Duplicate Values Function
def duplicate_values():
    print("Results of Duplicate Values")
    print("___________________________")
    try:
        for i, year in enumerate([2563, 2564, 2565, 2566]):
            duplicated = df_list[i].duplicated().sum()
            print(f"Report_{year}: ", duplicated)

        print("province Code: ", df_p_code.duplicated().sum())

    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")

duplicate_values()

Results of Duplicate Values
___________________________
Report_2563:  0
Report_2564:  0
Report_2565:  0
Report_2566:  0
province Code:  0


#### Step 3: Replace Values

In [ ]:
# Replace Values Function
def replace_values():
    # Loop through each DataFrame in the list
    for i in range(len(df_list)):
        try:
     
            # Create a copy to avoid modifying the original directly
            df = df_list[i].copy()

            # Replace ["จังหวัด"]
            df["จังหวัด"] = df["จังหวัด"].replace("ไม่ทราบ", "ไม่ระบุจังหวัด")
            # Remove row "รวม"
            #df = df[df["จังหวัด"] != "รวม"]

            #df = df.rename(columns = {'รวม': 'จำนวนผู้ป่วย'}, inplace = True)
            # Delete comma and convert to integer type
            for col in df.columns[1:20]:
                df[col] = (df[col].astype(str).str.replace(",","", regex = False)).astype("Int64")

            # Update the original list with the cleaned DataFrame
            df_list[i] = df
    
        except Exception as e:
            print(f"เกิดข้อผิดพลาด: {e}")

#df_list = [replace_values(df)for df in df_list]
replace_values()
print(" Results of Replace Values: Sample Dataset (Report 2563)")

df_list[0].head()

 Results of Replace Values: Sample Dataset (Report 2563)


,จังหวัด,โรคสมองเสื่อม,ติดแอลกอฮอล์,ติดยาบ้า (Amphetamine),ติดสารเสพติดอื่นๆ,โรคจิตเภท,โรคจิตอื่นๆ,โรคอารมณ์สองขั้ว,โรคชึมเศร้า,โรควิตกกังวล,...,ความบกพร่องทางการเรียนรู้,โรคออทิสติก,โรคสมาธิสั้น,พยามยามฆ่าตัวตาย (การตั้งใจทำร้ายตนเอง),ผู้ป่วยติดเกมส์ในผู้ใหญ่ (15 ปีขึ้นไป),ผู้ป่วยติดเกมส์ในเด็ก (อายุต่ำกว่า 15 ปี),โรคทางจิตเวชอื่นๆ,โรคลมชัก,รวม,ปี
0,กรุงเทพมหานคร,1018,365,359,468,1967,412,437,12215,2686,...,508,1195,2351,665,28,48,3600,1884,31340,2563
1,สมุทรปราการ,146,326,435,198,888,227,46,4839,1095,...,181,166,804,405,0,6,1631,741,12272,2563
2,นนทบุรี,630,468,429,787,1636,348,339,3917,2555,...,303,395,1162,343,7,12,4403,1410,19656,2563
3,ปทุมธานี,179,215,493,423,1439,409,84,3060,1603,...,139,224,440,179,4,3,3123,783,13013,2563
4,พระนครศรีอยุธยา,78,375,597,680,720,207,44,3332,878,...,57,60,134,240,2,0,1841,419,9716,2563


#### Step 4: Drop Columns

In [ ]:
# Drop Columns Function
def drop_columns(df):
    try:
        # Remove row "รวม"
        df = df[df["จังหวัด"] != "รวม"]
        # Drop columns index 1 to 18
        return df.drop(df.columns[1:19], axis = 1)
        
    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")

df_clean_list = [drop_columns(df) for df in df_list]

print("After dropping columns: sample dataset report_2563")
df_clean_list[0].head()

After dropping columns: sample dataset report_2563


,จังหวัด,รวม,ปี
0,กรุงเทพมหานคร,31340,2563
1,สมุทรปราการ,12272,2563
2,นนทบุรี,19656,2563
3,ปทุมธานี,13013,2563
4,พระนครศรีอยุธยา,9716,2563


#### Step 5: Merge DataFrame

Drop columns version

In [33]:
# Merge drop columns version
def drop_merge(df, df_p_code):
    
    # Merge DataFrame 2563, 2564, 2565, 2566
    df_merged = pd.concat(df, ignore_index = True)
    
    # Merge with province code
    df_merged_code = df_merged.merge(df_p_code, on = "จังหวัด", how = "left")
    
    # Fill missing province codes
    if "pro_code" in df_merged_code.columns:
        df_merged_code["pro_code"] = df_merged_code["pro_code"].fillna(0).astype("Int64")
    
    # Rename column "รวม" to "จำนวนผู้ป่วย"
    df_merged_code.rename(columns = {'รวม': 'จำนวนผู้ป่วย'}, inplace = True)

    return df_merged_code

df_drop_merged = drop_merge(df_clean_list, df_p_code)

print("After Merged Mental Health Data Overview (drop columns)")
df_drop_merged.head()

After Merged Mental Health Data Overview (drop columns)


,จังหวัด,จำนวนผู้ป่วย,ปี,pro_code
0,กรุงเทพมหานคร,31340,2563,10
1,สมุทรปราการ,12272,2563,11
2,นนทบุรี,19656,2563,12
3,ปทุมธานี,13013,2563,13
4,พระนครศรีอยุธยา,9716,2563,14


In [ ]:
# Export drop columns version
df_drop_merged.to_csv("data\\report_2563-2566.csv", index = False, encoding = "utf-8-sig")

Full merge version

In [34]:
# Merge Mental Health Datasets Function
def merge_data(df, df_p_code):
    try:
        # Merge DataFrame 2563, 2564, 2565, 2566
        df_merged = pd.concat(df, ignore_index = True)
        # Insert province code
        df_merged_code = df_merged.merge(df_p_code, on = "จังหวัด", how = "left")
        # Fillna
        if "pro_code" in df_merged_code.columns:
            df_merged_code["pro_code"] = df_merged_code["pro_code"].fillna(0).astype("Int64")
            
        return df_merged_code
        
    except Exception as e:
        print(f"เกิดข้อผิดพลาด: {e}")
         
df_merged = merge_data(df_list, df_p_code)

print("Merged Mental Health Data Overview (full merge)")
df_merged

Merged Mental Health Data Overview (full merge)


,จังหวัด,โรคสมองเสื่อม,ติดแอลกอฮอล์,ติดยาบ้า (Amphetamine),ติดสารเสพติดอื่นๆ,โรคจิตเภท,โรคจิตอื่นๆ,โรคอารมณ์สองขั้ว,โรคชึมเศร้า,โรควิตกกังวล,...,โรคออทิสติก,โรคสมาธิสั้น,พยามยามฆ่าตัวตาย (การตั้งใจทำร้ายตนเอง),ผู้ป่วยติดเกมส์ในผู้ใหญ่ (15 ปีขึ้นไป),ผู้ป่วยติดเกมส์ในเด็ก (อายุต่ำกว่า 15 ปี),โรคทางจิตเวชอื่นๆ,โรคลมชัก,รวม,ปี,pro_code
0,กรุงเทพมหานคร,1018,365,359,468,1967,412,437,12215,2686,...,1195,2351,665,28,48,3600,1884,31340,2563,10
1,สมุทรปราการ,146,326,435,198,888,227,46,4839,1095,...,166,804,405,0,6,1631,741,12272,2563,11
2,นนทบุรี,630,468,429,787,1636,348,339,3917,2555,...,395,1162,343,7,12,4403,1410,19656,2563,12
3,ปทุมธานี,179,215,493,423,1439,409,84,3060,1603,...,224,440,179,4,3,3123,783,13013,2563,13
4,พระนครศรีอยุธยา,78,375,597,680,720,207,44,3332,878,...,60,134,240,2,0,1841,419,9716,2563,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311,ปัตตานี,353,409,632,2785,3835,799,485,2843,5015,...,294,577,144,0,4,3151,1547,23634,2566,94
312,ยะลา,270,279,494,1281,2317,811,413,3269,4064,...,326,766,308,0,29,3346,1603,20157,2566,95
313,นราธิวาส,259,226,659,6472,3605,1288,283,4084,4513,...,322,712,249,2,27,3779,1881,29078,2566,96
314,ไม่ระบุจังหวัด,1075,3816,2475,4998,6911,2549,619,6062,10214,...,763,1699,794,6,18,13430,4199,61532,2566,0


In [40]:
# Export full merge 
df_merged.to_csv("data\\report_2563-2566_fullmerge.csv", index = False, encoding = "utf-8-sig")


## 3. Next Steps
- 🧼 **Refactor data cleaning steps** into modular functions for better clarity and reusability.
- 🗺️ **Visualize patient distribution** across provinces using a choropleth map, and 🔥 **highlight disease prevalence** with a heatmap.
- 📊 **Summarize key insights** using KPI cards, including total patients, highest and lowest reporting provinces.
- 📈 **Explore how the top 5 mental health conditions shift across years**, using year-wise filtering and bullet lists
- 📝 Wrap up final edits and share the dashboard via GitHub Pages.